In [22]:
import xml.etree.ElementTree as ET
from pathlib import Path
import shutil
import typing
from borb.pdf import Document, PDF
import os

class Xml_a_pdf:
    def __init__(self):
        self.datos_formulario = {}

    # Función para copiar y luego rellenar el PDF
    def rellenar_plantilla(self, plantilla, datos_formulario):
        carpeta_destino = './Reportes'
        # Verificar si la carpeta de destino existe, si no, crearla
        if not os.path.exists(carpeta_destino):
            os.makedirs(carpeta_destino)
            
        # Hacer una copia del archivo PDF
        a = shutil.copy(plantilla, carpeta_destino)
        a = Path(a)
        print(a)
        # Abrir la copia del PDF
        doc: typing.Optional[Document] = None
        with open(a, "rb") as pdf_file_handle:
            doc = PDF.loads(pdf_file_handle)
        assert doc is not None

        # Rellenar los campos del formulario en la copia
        doc.get_page(0).set_form_field_value("doc", datos_formulario['doc'])
        doc.get_page(0).set_form_field_value("nombre", datos_formulario['nombre'])
        doc.get_page(0).set_form_field_value("fecha", datos_formulario['fecha'])
        doc.get_page(0).set_form_field_value("documento", datos_formulario['documento'])
        doc.get_page(0).set_form_field_value("item", datos_formulario['item'])
        doc.get_page(0).set_form_field_value("codigo", datos_formulario['codigo'])
        doc.get_page(0).set_form_field_value("descripcion", datos_formulario['descripcion'])
        doc.get_page(0).set_form_field_value("und", datos_formulario['und'])
        doc.get_page(0).set_form_field_value("cantidad", datos_formulario['cantidad'])
        doc.get_page(0).set_form_field_value("vUnitario", datos_formulario['vUnitario'])
        doc.get_page(0).set_form_field_value("pUnitario", datos_formulario['pUnitario'])
        doc.get_page(0).set_form_field_value("valorV", datos_formulario['valorV'])
        doc.get_page(0).set_form_field_value("numeroTexto", datos_formulario['numeroTexto'])
        doc.get_page(0).set_form_field_value("opGravada", datos_formulario['opGravada'])
        doc.get_page(0).set_form_field_value("igv", datos_formulario['igv'])
        doc.get_page(0).set_form_field_value("total", datos_formulario['total'])
        doc.get_page(0).set_form_field_value("observacionesSunat", datos_formulario['observacionesSunat'])
        doc.get_page(0).set_form_field_value("direccionSucursal", datos_formulario['direccionSucursal'])
        doc.get_page(0).set_form_field_value("observacion", datos_formulario['observacion'])
        doc.get_page(0).set_form_field_value("placa", datos_formulario['placa'])
        doc.get_page(0).set_form_field_value("hash", datos_formulario['hash'])
        doc.get_page(0).set_form_field_value("ruc", datos_formulario['ruc'])

        # Guardar el PDF rellenado
        with open(a, "wb") as pdf_file_handle:
            PDF.dumps(pdf_file_handle, doc)
    
    def leer_XML(self, archivo):
        tree = ET.parse(archivo)
        root = tree.getroot()
        # Definir el namespace para cbc y cac
        namespaces = {
            'xmlns': "urn:oasis:names:specification:ubl:schema:xsd:Invoice-2",
            'biz': "urn:bizlinks:names:specification:ubl:peru:schema:xsd:BizlinksAggregateComponents-1",
            'cac': "urn:oasis:names:specification:ubl:schema:xsd:CommonAggregateComponents-2",
            'cbc': "urn:oasis:names:specification:ubl:schema:xsd:CommonBasicComponents-2",
            'ds': "http://www.w3.org/2000/09/xmldsig#",
            'ext': "urn:oasis:names:specification:ubl:schema:xsd:CommonExtensionComponents-2",
            'qdt': "urn:oasis:names:specification:ubl:schema:xsd:QualifiedDatatypes-2",
            'sac': "urn:sunat:names:specification:ubl:peru:schema:xsd:SunatAggregateComponents-1"
        }

        # Fecha y Hora
        date = root.find("cbc:IssueDate", namespaces).text
        time = root.find("cbc:IssueTime", namespaces).text
        # datetime.strptime(date.text, '%Y-%m-%d')
        fechaEmision = f"{date} {time}"


        # Código Hash
        hash = root.find('.//ds:DigestValue', namespaces).text

        # Nombre
        ## Buscar el valor de cbc:RegistrationName dentro de cac:AccountingCustomerParty
        nombre = root.find('.//cac:AccountingCustomerParty//cbc:RegistrationName', namespaces).text

        if nombre is not None:
            print(nombre)  # Esto imprimirá 'CLIENTES VARIOS'
        else:
            print("No se encontró el valor de cbc:RegistrationName en AccountingCustomerParty")
            
        # DNI
        documento = root.find('.//cac:AccountingCustomerParty//cbc:ID', namespaces).text
        print (documento)
        # Documento boleta, factura etc
        doc = root.find('cbc:ID', namespaces)
        doc = doc.text
        print(doc)
        # Items
        ## Item
        item = root.find('.//cac:InvoiceLine//cbc:ID', namespaces).text
        print(item)

        ## Código
        codigo = root.find('.//cac:InvoiceLine//cac:Item//cbc:ID', namespaces).text
        print(codigo)

        ## Descripción
        desc = root.find('.//cac:InvoiceLine//cac:Item//cbc:Description', namespaces).text

        ## Und.
        und = root.find('.//cac:InvoiceLine//cbc:InvoicedQuantity', namespaces)
        und = und.attrib.get('unitCode')
        print(f'und: {und}')

        ## Cantidad
        cantidad = root.find('.//cac:InvoiceLine//cbc:InvoicedQuantity', namespaces).text
        cantidad = format(float(cantidad), '.2f')
        print(f'cantidad: {cantidad}')

        ## V. Unitario
        vUnitario = root.find('.//cac:InvoiceLine//cbc:LineExtensionAmount', namespaces).text
        vUnitario = format(float(vUnitario), '.4f')
        print(f'vUnitario: {vUnitario}')

        ## P. Unitario
        pUnitario = root.find('.//cac:InvoiceLine//cac:PricingReference//cbc:PriceAmount', namespaces).text
        pUnitario = format(float(pUnitario), '.2f')
        print(f'pUnitario: {pUnitario}')

        ## Valor Venta
        valorV = root.find('.//cac:InvoiceLine//cac:Price//cbc:PriceAmount', namespaces).text
        print(f'valorV: {valorV}')
        # Moneda
        opGravada = root.find('.//cac:TaxTotal//cac:TaxSubtotal//cbc:TaxableAmount', namespaces).text
        print(f'opGravada; {opGravada}')

        porcentaje = root.find('.//cac:TaxTotal//cac:TaxCategory//cbc:Percent', namespaces).text
        porcentaje = f"{porcentaje[:2]}%"

        igv = root.find('.//cac:TaxTotal//cac:TaxSubtotal//cbc:TaxAmount', namespaces).text
        igv = format(float(igv), '.2f')
        print(f'igv: {igv}')

        total = root.find('.//cac:LegalMonetaryTotal//cbc:PayableAmount', namespaces).text
        total = format(float(total), '.2f')
        print(f'total: {total}')


        # Numero en texto
        numeroTexto = root.find('cbc:Note', namespaces).text

        # Observaciones
        archivoR = f"R-{Path(archivo).stem}"
        treeR = ET.parse(f"./{archivoR}/{archivoR}.xml")
        rootR = treeR.getroot()
        observaciones = rootR.find('.//cac:DocumentResponse//cbc:Description', namespaces).text
        print(f'observaciones: {observaciones}')
        
        # Informacion Adicional
        additional_properties = root.findall('.//ext:ExtensionContent//biz:AdditionalProperty', namespaces)

        additional_data = []
        direccionSucursal = '0'
        observacion = '0'
        placa = '0'
        for prop in additional_properties:
            id_value = prop.find('cbc:ID', namespaces).text
            value = prop.find('cbc:Value', namespaces).text
            additional_data.append({'ID': id_value, 'Value': value})
            
        for i in additional_data:
            id_value = int(i['ID'])
            value = i['Value']
            
            # Direccion de la Sucursal
            if id_value == 9785:
                direccionSucursal = value
            # Placa
            elif id_value == 9114:
                observacion = value
            # Observación
            elif id_value == 9618:
                placa = value
                
        print(placa)
        print(direccionSucursal)
        print(observacion)
        
        # R.U.C
        ruc = root.find('cac:Signature//cac:SignatoryParty//cbc:ID', namespaces).text
        ruc = f'{ruc}-{doc}'
        
        self.datos_formulario = {'fecha': fechaEmision, 'nombre': nombre, 'doc': doc, 'documento': documento, 'item': item, 'codigo': codigo, 
                                 'descripcion': desc, 'und': und, 'cantidad': cantidad, 'vUnitario': vUnitario, 'pUnitario': pUnitario, 
                                 'valorV': valorV, 'numeroTexto': numeroTexto, 'opGravada': opGravada, 'igv': igv, 'total': total, 'observacionesSunat': observaciones,
                                 'direccionSucursal': direccionSucursal, 'observacion': observacion, 'placa': placa, 'hash': hash, 'ruc': ruc}

In [14]:
from borb.pdf import Document
from borb.pdf import PDF


def rellenar_plantilla(plantilla):
    # open document
    doc: typing.Optional[Document] = None # type: ignore
    with open(plantilla, "rb") as pdf_file_handle:
        doc = PDF.loads(pdf_file_handle)
    assert doc is not None

    # set
    doc.get_page(0).set_form_field_value("doc", doc)
    doc.get_page(0).set_form_field_value("nombre", registration_name_customer)
    doc.get_page(0).set_form_field_value("fecha", fechaEmision)
    doc.get_page(0).set_form_field_value("documento", documento)
    doc.get_page(0).set_form_field_value("item", item)
    doc.get_page(0).set_form_field_value("codigo", codigo)
    doc.get_page(0).set_form_field_value("descripcion", desc)
    doc.get_page(0).set_form_field_value("und", und)
    doc.get_page(0).set_form_field_value("cantidad", cantidad)
    doc.get_page(0).set_form_field_value("vUnitario", vUnitario)
    doc.get_page(0).set_form_field_value("pUnitario", pUnitario)
    doc.get_page(0).set_form_field_value("valorV", valorV)
    doc.get_page(0).set_form_field_value("numeroTexto", numeroTexto)
    doc.get_page(0).set_form_field_value("opGravada", opGravada)
    doc.get_page(0).set_form_field_value("igv", igv)
    doc.get_page(0).set_form_field_value("total", total)
    doc.get_page(0).set_form_field_value("observacionesSunat", observaciones)
    doc.get_page(0).set_form_field_value("direccionSucursal", direccionSucursal)
    doc.get_page(0).set_form_field_value("observacion", observacion)
    doc.get_page(0).set_form_field_value("placa", placa)
    doc.get_page(0).set_form_field_value("hash", hash)
    doc.get_page(0).set_form_field_value("ruc", ruc)
    # store
    with open("./plantillas/20517252558-03-B311-01759541.pdf", "wb") as pdf_file_handle:
        PDF.dumps(pdf_file_handle, doc)

In [23]:
clase = Xml_a_pdf()
plantilla = './plantillas/boleta.pdf'
archivo = './20517252558-03-B111-00485883.xml'
clase.leer_XML(archivo)


CLIENTES VARIOS
11111111
B111-00485883
1
1
und: NIU
cantidad: 1.00
vUnitario: 4.2400
pUnitario: 5.00
valorV: 4.24
opGravada; 4.24
igv: 0.76
total: 5.00
observaciones: La boleta número B111-00485883, ha sido aceptada
0
0
0


In [3]:
clase.datos_formulario

{'fecha': '2024-10-01 00:27:48',
 'nombre': 'CLIENTES VARIOS',
 'doc': 'B111-00485883',
 'documento': '11111111',
 'item': '1',
 'codigo': '1',
 'descripcion': 'Cat: 1',
 'und': 'NIU',
 'cantidad': '1.00',
 'vUnitario': '4.2400',
 'pUnitario': '5.00',
 'valorV': '4.24',
 'numeroTexto': 'Son: Cinco con 00/100. Soles',
 'opGravada': '4.24',
 'igv': '0.76',
 'total': '5.00',
 'observacionesSunat': 'La boleta número B111-00485883, ha sido aceptada',
 'direccionSucursal': '0',
 'observacion': '0',
 'placa': '0',
 'hash': 'Zg80TZtYTH2wS2br68sm3lz+VaE=',
 'ruc': '20517252558-B111-00485883'}

In [26]:
clase.rellenar_plantilla(plantilla, clase.datos_formulario)

Reportes\boleta.pdf
